In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv
/kaggle/input/train/train.csv


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
train_data = pd.read_csv('/kaggle/input/students-drop-out-prediction/train.csv')
test_data = pd.read_csv('/kaggle/input/students-drop-out-prediction/test.csv')

In [4]:
keys = list(train_data.keys())
X_train, y_train = train_data[keys[1:-1]], train_data['label']

In [5]:
X_test = test_data[keys[1:-1]]

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [7]:
estimators = [
    ('rf', RandomForestClassifier(random_state=8, max_features = 25)),
    ('gb', GradientBoostingClassifier(random_state=8)),
    ('xgb', XGBClassifier(random_state=8))
]

sclf = StackingClassifier(
    estimators=estimators,
    final_estimator=MLPClassifier()
)

fitted_sclf = sclf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [8]:
y_pred = fitted_sclf.predict(X_train)

In [9]:
fitted_sclf.score(X_train, y_train)

0.934931506849315

In [10]:
test_pred = fitted_sclf.predict(X_test)

In [11]:
df = pd.DataFrame(range(len(test_pred)), columns=['id'])
df['label'] = test_pred
df

,id,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1623,1623,0
1624,1624,1
1625,1625,0
1626,1626,2


In [12]:
df.to_csv('submission.csv', index=False)